# ***<font color="RoyalBlue">アセンブリ・データQC（grape_qc_assembly）</font>***

カバレッジの計算、アセンブリ（SPAdesまたはSKESA）、quality check (checkM) を行う。

<br>
<br>

---

# ***<u><font color="RoyalBlue">解析手順</font></u>***


## 1. 入力ファイルのアップロード

解析プロジェクトのフォルダへ、FASTQファイルをアップロードしてください。  
左サイドバー上部の⬆︎ボタンから、または、解析プロジェクトフォルダを開きファイルをドラック＆ドロップでアップロードできます。
<br>


使用できるファイルの拡張子は、`fastq.gz`です。<br>
ファイル名は、菌株名で始まるようにしてください。<br>
たとえば、以下のようなフォーマットに対応しています。

- イルミナフォーマット（e.g., strain_S00_L001_R1_001.fastq.gz）
- シンプルフォーマット（e.g., strain_R1.fastq.gz）

## 2. 菌株リストの作成

次のいずれかの方法で、菌株リストを作成します。

### 2.1. Jupyter Labで作成する
- 左サイドバー上部の「 + 」ボタン→「Other」の「Text file」を押下し、ファイルを新規作成できる。
- 編集が完了したら、Ctrl＋s（Mac:Command+s）を押下し、ファイル名を設定して保存する。
### 2.2.  Notepad、メモ帳、Vim等(何でも良い)で作成して、解析フォルダにコピーする。
- Windowsで作成すると、正常に動かない可能性があります。（改行コードの違いのため）
- LinuxまたはMacで作成してください。
### 2.3. 下記コマンドで作成する。       
- 「file_name = 」の後に、リスト名をダブルクオート(")で囲って入力してください（例, "ファイル名") 
- 「user_input = ("""」の後に、菌株リストを入力してください（1行に1つの株名）。

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font>

In [ ]:
# リストのファイル名を入力
file_name = "list.txt"

# 菌株リストを貼り付ける
user_input = ("""
A0001
A0002
A0003
A0004
A0005
A0006
A0007
A0008
"""
)

# 前後の改行を除去後に改行追加
user_input = user_input.strip("\n") + "\n"

# 入力された要素をファイルに書き込む
with open(file_name, "w") as file:
    file.write(user_input)
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>
<br>

## 4. FASTQリストの作成

3.で作成した菌株リスト中の各菌株に紐づくFASTQファイル名(ショートリードデータのファイル名)一覧の作成を以下の手順で行います。

### 4.1. find_strain_pairs.pyの実行
- 「file_name = 」の後に、前の項で作成した株名リスト名をダブルクオート(")で囲って入力してください。 
- `パラメータを設定`に記載されている内容で修正したいものがあれば修正し、実行してください。
    - 記載されている内容のまま実行することも可能です。

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font>

In [ ]:
import subprocess

####################################################
# パラメータを設定
####################################################
# 2.で作成した菌株リストのファイル名を入力
file_name = "list.txt"
# FASTQファイルの拡張子を入力
file_extension = "fastq.gz"
# 作成するFASTQリストのファイル名
fastq_list_name = "list_fastq.tsv"
# ペアが見つからなかった菌株一覧のファイル名
unpaired_list = "unpaired_fastq.tsv"

####################################################
# Run find_strain_pairs.py
# ※以下は変更しないでください※
####################################################
command = [
    "find_strain_pairs.py", file_name,
    "--file_extension", file_extension,
    "--paired_list", fastq_list_name,
    "--unpaired_list", unpaired_list
]
subprocess.run(command, capture_output=False, text=True)
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>
<br>

### 4.2. FASTQリストの確認・修正
- `list_fastq.tsv`(FASTQリスト)を開いて確認し、各菌株名に対し2つのFASTQファイル(Read1と2)名が正しく記載されていることを確認します。
- `unpaired_fastq.tsv`が生成された場合は開いて確認し、FASTQファイルのペアが見つからなかった菌株を確認してください。
    - `list_fastq.tsv`にその菌株名をコピーして、タブ区切りでFASTQファイルのペアを記載してください。
- FASTQリストの記載例は以下の通りです(各列はタブ区切り、改行コードは\n)。

```plaintxt
strain1	strain1_R1.fastq.gz	strain1_R2.fastq.gz
strain2	strain2_R1.fastq.gz	strain2_R2.fastq.gz
```

## 5. Configuration file（設定ファイル）を作製する<br>

<b>QCが必要ない場合は、次のステップ6に行ってください。</b><br>
<br>

QCを行う場合は、次の通りconfiguration.txtの作成を行ってください。<br>

Create Projectで使った`CreateProject.ipynb`ファイルが配置されたフォルダにある`sample_data`の中に、`ehec/configuration.txt`というファイルがあります。これがEHEC用のサンプル設定ですので、これをプロジェクトフォルダにコピーして、解析する菌種に応じて適宜編集してください。<br>

configuration.txtでは、ゲノムサイズ、必要カバレッジを指定して、データ量が十分か判断します。<br>
ゲノムサイズは菌種ごとに異なるので、適宜指定する必要があります。<br>
チェックが不要な項目は、その行を削除すればチェックは行いません。<br>
<br>
以下にEHECの例を挙げます。<br>
<br>
Genome size (Mbp):	5.5 <br>
Coverage threshold:	40 <br>
Maximum number of contigs:	1000 <br>
Minimum genome size (Mbp):	4 <br>
Maximum genome size (Mbp):	6.5 <br>
Minimum completeness (%):	98 <br>
Maximum contamination (%):	2 <br>


## 6. 	Grapeを実行する（データ量チェック、アセンブリ）

環境に合わせて、4.1.か4.2.のどちらのgrapeを実行するかお選びください。

### 6.1. マシンのメモリが40GB以上の場合
<font color="Tomato">マシンのメモリが40GB以上の場合 </font> <br>

下記セル内のリスト部分（「list = "list_fastq.tsv"」のダブルクオートで囲まれた部分）を作成したリスト名に変えて、実行する。

|変数|パラメータ|必須か|概要|
|---|---|---|---|
|list|-i|●|4.で作成したFASTQリストのファイル名。|
|assembler|-a|-|使用するassembler [p:SPAdes, k:SKESA]	( p by default )|
|config|-c|-|grape中で品質判定を行う際の閾値設定ファイル。指定しない場合、品質判定を行わない。|
|cleanup|-u|-|中間ファイルを削除するか否か。[1: cleanup, 0: not cleanup] ( 1 by default )|
|cutoff length|-L|-|指定した長さ未満のコンティグを除去する。( 500 by default )|
|fastp|-p|-|fastpによるクオリティトリミング(クオリティの低い塩基を除去)を実行するか否か。 [1:run, 0:not run]( 1 by default )|
|scaffold|-s|-|Scaffold配列を作成するか否か。assemblerにSPAdesを指定した場合のみ有効。[1: create, 0: not create] (0 by default)|
|threads|-t|-|スレッド数。No. of Thread( 8 by default )|

<br>

<font color="Tomato">────────────── 　↓↓↓ 　***コマンド実行***　 ↓↓↓　 ──────────────</font><br>
※ ログは、実行ディレクトリの`grape_qc_assembly.log`を参照してください。（<u><font color="Red">***実行のたびに上書きされます。***</u></font>）

In [ ]:
####################################################
# パラメータを設定
####################################################
# 菌株リスト
list = "list_fastq.tsv"
# 設定ファイル
# 品質判定を行いたい場合、品質設定ファイルを作成して以下にファイル名を指定してください。
config = "" 
# config = "configuration.txt"
# アセンブラー： SPAdesを用いる場合は「p」, SKESAを用いる場合は「k」を入力
assembler = "p"
# コンティグの閾値：この値未満の長さ (bp)のコンティグは削除されます
cutoff = 500
# fastpによるトリミングを行う場合は「1」を、使用しない場合は「0」を入力
fastp = 1
# スキャフォールドを作製する場合は「1」を、作製しないしない場合は「0」を入力
scaffold = 0
# スレッド数
threads = 8
# トリミング後fastqなど中間ファイルを残す場合には「0」を入力。削除する場合は「1」を入力。
cleanup = 1

####################################################
# Run grape_qc_assembly (変更不要)
####################################################
# Run grape_qc_assembly
config_option = f"-c {config} " if config else ""

!bash grape_qc_assembly.sh \
    -i $list \
    -a $assembler \
    -L $cutoff \
    -p $fastp \
    -s $scaffold \
    -t $threads \
    -u $cleanup $config_option\
    > grape_qc_assembly.log
!echo 'Complete!'

### 6.2. マシンのメモリが40GB以下の場合
<font color="Tomato">※マシンのメモリが40GB 以下の場合 </font>

下記セル内のリスト部分（「list = "list_fastq.tsv"」のダブルクオートで囲まれた部分）を作成したリスト名に変えて、実行する。<br>
※各パラメータの詳細は、`6.1. マシンのメモリが40GB以上の場合`をご覧ください。

In [ ]:
####################################################
# パラメータを設定
####################################################
# 菌株リスト
list = "list_fastq.tsv"
# 設定ファイル
# 品質判定を行いたい場合、品質設定ファイルを作成して以下にファイル名を指定してください。
config = "" 
# config = "configuration.txt"
# アセンブラー： SPAdesを用いる場合は「p」, SKESAを用いる場合は「k」を入力
assembler = "p"
# コンティグの閾値：この値未満の長さ (bp)のコンティグは削除されます
cutoff = 500
# fastpによるトリミングを行う場合は「1」を、使用しない場合は「0」を入力
fastp = 1
# スキャフォールドを作製する場合は「1」を、作製しないしない場合は「0」を入力
scaffold = 0
# スレッド数
threads = 8
# トリミング後fastqなど中間ファイルを残す場合には「0」を入力。削除する場合は「1」を入力。
cleanup = 1

####################################################
# Run grape_qc_assembly (変更不要)
####################################################
# Run grape_qc_assembly
config_option = f"-c {config} " if config else ""

!bash grape_qc_assembly.sh \
    -i $list \
    -a $assembler \
    -L $cutoff \
    -p $fastp \
    -s $scaffold \
    -t $threads \
    -u $cleanup $config_option \
    -r 1 \
    > grape_qc_assembly.log
!echo 'Complete!'

<font color="Tomato">────────────────────────────────────────────</font>

<br>

### 5.	実行後、`qc_assembly_[日付]_[時間]_[リスト名]`のフォルダが作製される。フォルダ内のファイルは次の通り<br>

- [株名].fasta：ドラフトゲノム
- coverage.txt：カバレッジ一覧
- list_over_coverage：設定ファイルで指定したカバレッジを上回った株のリスト。このリストにある株のみをアセンブルする。
- list_assembly.tsv：list_over_coverageに含まれる菌株のFASTQファイルリスト。
- assembly_summary.tsv：コンティグの長さなどの統計（QUASTで計算）をまとめたもの。
- qc_results.tsv：QUAST, CheckMの結果（ゲノムサイズ、コンタミネーションなど）から設定ファイルの範囲内にある場合にpass, 外れている場合にfailを記載する。全ての値でpassとなると、qc_resultsの列がPASSとなる。
- qc_results.xlsx：qc_resutls.tsvのエクセル版（内容は同じ）
- cleanup=0の時のみ残るデータ
  - fastpフォルダ：fastpでトリミングしたfastqファイル
  - quastフォルダ：QUASTの実行結果
  - checkmフォルダ：CheckMの実行結果

基本的には、PASSしたデータをその後の解析（SNP解析等）に用いる。<br>
ただし、Quality checkにpassしなかったデータも、全く使えないとも限らない。明らかなコンタミ等でなければ、データを精査する。